In [2]:
import  numpy as np
import os 
import onnxruntime as ort
import torch
import matplotlib.pyplot as plt
import cv2

In [3]:
from ultralytics import YOLO



In [4]:
model = YOLO('/home/leloc/Documents/VAST/Skin_segmentation/best_model.pt')

In [5]:
results = model.predict(source="/home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images", save=True)  # Thay đổi 'path/to/test/images' tới thư mục chứa ảnh kiểm tra
for result in results:
    print(result)  # Chứa thông tin các bounding box, nhãn, và độ tin cậy cho mỗi ảnh

# Đánh giá model (Precision, Recall, mAP, v.v.) trên tập kiểm tra
metrics = model.val(data="/home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/data.yaml")  # data.yaml chứa thông tin về các lớp và đường dẫn dữ liệu
print(metrics)  # 


image 1/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_1021_112_40_ACK_png.rf.c894eb43967394a442b76e29241d63ce.jpg: 640x640 (no detections), 5.9ms
image 2/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_1022_114_756_ACK_png.rf.28e3cb5bdcd437e912e74c9730a10278.jpg: 640x640 1 ACK, 6.0ms
image 3/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_1027_128_854_ACK_png.rf.3005a319ef1f7d9bf51e1454113edf3b.jpg: 640x640 (no detections), 5.9ms
image 4/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_1031_148_66_ACK_png.rf.b7b7e66b983570d59ca32c21d5f63208.jpg: 640x640 1 ACK, 5.9ms
image 5/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_1042_187_746_ACK_png.rf.0a5cf8b947b8f79252e1d1eb41f697ca.jpg: 640x640 (no detections), 6.0ms
image 6/228 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v

val: Scanning /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/valid/labels.cache... 228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 228/228 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:03<00:00,  4.63it/s]


                   all        228        317      0.316      0.415      0.353      0.126      0.487      0.445      0.433      0.196
                   ACK         74        135      0.304      0.311      0.232     0.0795      0.306      0.222      0.182     0.0646
                   BCC         81         87      0.329      0.552      0.404      0.147      0.494      0.621      0.519      0.194
                   MEL          9         12      0.257      0.167      0.209     0.0655      0.701      0.393       0.47      0.127
                   NEV         26         30      0.498      0.667      0.613      0.206      0.565      0.667      0.683      0.395
                   SCC         17         18      0.111      0.111      0.196      0.102      0.387      0.167      0.249      0.143
                   SEK         21         35      0.397      0.686      0.463      0.158      0.468      0.604      0.493      0.249
Speed: 1.3ms preprocess, 6.1ms inference, 0.0ms loss, 0.7ms postproce

In [13]:
results = model.predict(source="/home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_29_40_561-png_NEV_png.rf.29453c8ff85f9299a237f57e0f58ba48.jpg")  # Thay đổi 'path/to/test/image.jpg' với đường dẫn ảnh của bạn

# Load ảnh để hiển thị
image = cv2.imread("/home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_29_40_561-png_NEV_png.rf.29453c8ff85f9299a237f57e0f58ba48.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Chuyển ảnh từ BGR sang RGB để hiển thị đúng màu

for result in results:
    if result.masks is not None:
        mask_array = result.masks.cpu().numpy()  # Chuyển toàn bộ mask sang numpy
        
        # Kiểm tra và thay đổi kích thước nếu cần
        for mask in mask_array:
            if mask.shape != image.shape[:2]:
                mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
            
            # Tạo ảnh màu cho mask
            colored_mask = np.zeros_like(image)
            colored_mask[mask > 0.5] = [255, 0, 0]  # Tô màu đỏ cho mask
            
            # Áp dụng mask lên ảnh gốc
            image = cv2.addWeighted(image, 1, colored_mask, 0.5, 0)

# Hiển thị và lưu ảnh
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.savefig('test.jpg')
plt.show()


image 1/1 /home/leloc/Documents/VAST/Skin_segmentation/skin disease.v2i.yolov8/test/images/PAT_29_40_561-png_NEV_png.rf.29453c8ff85f9299a237f57e0f58ba48.jpg: 640x640 1 SEK, 6.0ms
Speed: 9.2ms preprocess, 6.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


<Figure size 640x480 with 1 Axes>

In [7]:
import torch

# Kiểm tra số GPU có trên hệ thống
print("Số GPU có sẵn:", torch.cuda.device_count())

# Kiểm tra tên GPU
print("Tên GPU:", torch.cuda.get_device_name(0))

# Kiểm tra dung lượng VRAM
print("Tổng dung lượng VRAM (GB):", torch.cuda.get_device_properties(0).total_memory / (1024 ** 3), "GB")
print("Bộ nhớ đang sử dụng (MB):", torch.cuda.memory_allocated(0) / (1024 ** 2), "MB")
print("Bộ nhớ còn trống (MB):", torch.cuda.memory_reserved(0) / (1024 ** 2), "MB")

Số GPU có sẵn: 1
Tên GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Tổng dung lượng VRAM (GB): 3.80389404296875 GB
Bộ nhớ đang sử dụng (MB): 48.6376953125 MB
Bộ nhớ còn trống (MB): 1512.0 MB
